In [ ]:
#@title License and Disclaimer

'''
MIT License

Copyright (c) 2023 Parham Faraji

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
'''

In [2]:
#@title Data Import

import tensorflow as tf
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
train = tfds.load('wine_quality/white', split='train', shuffle_files=False)


In [3]:
#@title Packaging Function

import numpy as np
import cv2
from tqdm import tqdm

def normalize(cluster):
  clusterNorm = np.zeros_like(cluster)

  for f in range(cluster.shape[1]):
    offset = cluster[:,f] - np.min(cluster[:,f])
    clusterNorm[:,f] = offset / np.max(offset)
    del offset

  print(' min:', np.min(clusterNorm, axis=0), '\n max:',
                 np.max(clusterNorm, axis=0), '\n')
  del cluster
  return np.reshape(clusterNorm, [clusterNorm.shape[0],
                                  clusterNorm.shape[1], 1])

def package(data, count):
  # dataset = dataset.take(3)
  # list(dataset.as_numpy_iterator())
  given = []
  regress = []
  for sample in tqdm(data.take(count)):

    features = sample['features']

    given.append([
    features['alcohol'],
    features['chlorides'],
    features['citric acid'],
    features['density'],
    features['free sulfur dioxide'],
    features['pH'],
    features['residual sugar'],
    features['sulphates'],
    features['total sulfur dioxide'],
    features['volatile acidity']
    ])

    regress.append([
    sample['quality'],
    features['fixed acidity']
    ])

  given_out = np.array(given)
  regress_out = np.array(regress)
  del given, regress
  print(given_out.shape, regress_out.shape, '\n')
  return given_out, regress_out


In [4]:
#@title Packaging Data
# del V, Yv, X, Yx
giv, reg = package(train, 4898) #4898
giv = normalize(giv)
reg = normalize(reg)

split = 898
X, Yx, V, Yv= giv[split:], reg[split:], giv[:split], reg[:split]

Yv = np.reshape(Yv, [Yv.shape[0], Yv.shape[1]])
Yx = np.reshape(Yx, [Yx.shape[0], Yx.shape[1]])

print(X.shape, V.shape)
print(Yx.shape, Yv.shape)

100%|██████████| 4898/4898 [00:02<00:00, 1894.95it/s]


(4898, 10) (4898, 2) 

 min: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
 max: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] 

 min: [0. 0.] 
 max: [1. 1.] 

(4000, 10, 1) (898, 10, 1)
(4000, 2) (898, 2)


In [5]:
#@title Layer Designer (1D) Function

from tensorflow import keras

# This approach helps make design intuitive, intelligible, and easy to alter
# No need to worry about the order of operations
# You simply enable certain sub-layers and focus on the grand architecture
# It is especially useful as a tool for beginners to start from the middle by fiddling around
# The "engage" variable in the function is responsible for enabling all operations needed for a composite layer
# Using default parameters, layers can be on autopilot by verifying input/"consign" alone
def layer(consign, engage=[1,0,0,1,0,0], fmaps=24, k=3, act='relu', step=1, pool=2, scale=2, dp=.20, ep=1e-5, pad='same'): # same/valid

  # preset-choices:
  if engage[0]==1:
    consign = keras.layers.Conv1D(filters=fmaps, kernel_size=k, strides=step, padding=pad)(consign)
  if engage[0]==2:
    consign = keras.layers.Conv1DTranspose(filters=fmaps, kernel_size=k, strides=step, padding=pad)(consign)
  if engage[0]==3:
    attn = keras.layers.MultiHeadAttention(num_heads=3, key_dim=fmaps)
    consign = attn(consign, consign)

  if engage[1]:
    consign = keras.layers.BatchNormalization()(consign)
  if engage[2]:
    consign = keras.layers.LayerNormalization(epsilon=ep)(consign)
  if engage[3]:
    consign = keras.layers.Activation(act)(consign)
  if engage[4]:
    consign = keras.layers.MaxPooling1D(pool)(consign)
  if engage[5]:
    consign = keras.layers.UpSampling1D(scale)(consign)
  if engage[6]:
    consign = keras.layers.Dropout(dp)(consign)

  return consign


In [12]:
#@title 1D Regression Model Demo

'''1-dimensionals are cost efficient, so here we went with a depth of 7-8 layers, and width of 3-4, with as much demonstrated diversity'''
def architect(nc=2): # nc = number of output values

  # lunch
  inputshape = [10,1]
  inputlayer = keras.layers.Input(inputshape)

  # block a @lvl1
  a = layer(inputlayer, [1, 1,0, 1, 0,0,0], fmaps=30, k=3, act='relu')
  a = layer(a,          [1, 1,0, 1, 0,0,0], fmaps=30, k=3, act='relu')
  a = layer(a,          [1, 1,0, 1, 0,0,0], fmaps=30, k=2, act='relu')

  # block b @lvl1
  b = layer(inputlayer, [0, 0,0, 0, 0,1,0], scale=2)
  b = layer(b,          [1, 1,0, 1, 0,0,0], fmaps=30, k=5, act='relu', pad='valid')
  b = layer(b,          [1, 1,0, 1, 0,0,1], fmaps=30, k=4, act='relu', pad='valid')
  b = layer(b,          [1, 1,0, 1, 0,0,0], fmaps=30, k=4, act='relu', pad='valid')

  # block c @lvl1
  c = layer(inputlayer, [2, 0,0, 1, 0,0,0], fmaps=30, k=4, step=2, act='relu')
  c = layer(c,          [1, 0,0, 1, 0,0,0], fmaps=30, k=4, act='relu')
  c = layer(c,          [1, 0,0, 1, 1,0,0], fmaps=30, k=4, pool=2, act='relu')

  # res=inputlayer

  # merge
  con = keras.layers.Concatenate()([inputlayer, a, b, c])
  con = keras.layers.BatchNormalization()(con)

  # attention bypass
  a2 = layer(b,     [3, 0,1, 0, 0,0,0], fmaps=30)
  a2 = layer(a2,    [1, 1,0, 1, 0,0,1], fmaps=20, k=3, act='relu')

  # 2nd lvl independent layers
  b2 = layer(con,   [1, 0,0, 1, 0,0,0], fmaps=40, k=5, act='relu')

  c2 = layer(con,   [1, 1,0, 1, 0,0,0], fmaps=40, k=4, act='relu')

  d2 = layer(con,   [2, 1,0, 1, 0,0,0], fmaps=40, k=4, act='relu')

  # res 2
  res2 = layer(con, [1, 1,0, 1, 0,0,0], fmaps=30, k=1, act='linear')

  # merge 2
  con2 = keras.layers.Concatenate()([res2, a2, b2, c2, d2])
  con2 = keras.layers.BatchNormalization()(con2)

  # 3rd lvl independent layers
  a3 = layer(con2, [2, 1,0, 1, 0,0,0], fmaps=40, k=4, act='relu')

  b3 = layer(con2, [1, 1,0, 1, 0,0,0], fmaps=40, k=5, act='relu')

  c3 = layer(con2, [1, 1,0, 1, 0,0,0], fmaps=40, k=3, act='relu')

  d3 = layer(b3,   [1, 0,0, 1, 0,0,1], fmaps=40, k=4, act='relu')

  # res 3
  res3 = layer(con2, [1, 1,0, 1, 0,0,0], fmaps=30, k=1, act='linear')

  # merge 3
  con3 = keras.layers.Concatenate()([res3, a3, b3, c3, d3])
  con3 = keras.layers.BatchNormalization()(con3)

  # finalization
  f = layer(con3, [1, 1,0, 1, 0,0,0], fmaps=50, k=3, act='relu')
  f = layer(f,    [1, 1,0, 1, 0,0,0], fmaps=60, k=3, act='relu')

  # res 4
  res4 = layer(con3, [1, 1,0, 1, 0,0,0], fmaps=40, k=1, act='linear')

  # merge 4
  con4 = keras.layers.Concatenate()([res4, f])
  # con4 = keras.layers.BatchNormalization()(con4)

  gap = keras.layers.GlobalAveragePooling1D()(con4)
  outputlayer = keras.layers.Dense(nc, activation='sigmoid')(gap)

  return inputlayer, outputlayer


# tie regression model
input, output = architect()
model = keras.models.Model(inputs=input, outputs=output)

# compile model
model.compile(loss='mae', optimizer=keras.optimizers.Adam())

# print model
model.summary()

# implement callbacks
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.8,
                patience=5, min_lr=1e-4)

es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto',
                verbose=1, patience=35, restore_best_weights=True)

cp = keras.callbacks.ModelCheckpoint(
                './best_checkpoint', monitor="val_loss", verbose=0,
                save_best_only=True, save_weights_only=False, mode="auto",
                save_freq="epoch", initial_value_threshold=0.0487) # *threshold based on a previous test run

# fit data into model
model.fit(x=X, y=Yx, batch_size=308, epochs=270, verbose=1, # *batch size based on available hardware
                validation_data=(V, Yv), shuffle=True,
                callbacks=[reduce_lr, es, cp]) # or [reduce_lr, es]

# Save model
model.save('NTR')

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 10, 1)]              0         []                            
                                                                                                  
 up_sampling1d_4 (UpSamplin  (None, 20, 1)                0         ['input_5[0][0]']             
 g1D)                                                                                             
                                                                                                  
 conv1d_79 (Conv1D)          (None, 16, 30)               180       ['up_sampling1d_4[0][0]']     
                                                                                                  
 batch_normalization_86 (Ba  (None, 16, 30)               120       ['conv1d_79[0][0]']     

In [13]:
#@title Sample Prediction with Best Weights

from tensorflow.keras.models import load_model
'''either load or use existing model'''
modelcp = load_model('/content/best_checkpoint')
scroll = 32
predictions = modelcp.predict(V[::898//scroll])

2/2 [==============================] - 1s 139ms/step


In [14]:
for n in range(scroll):
  print(f'Predicted:{np.round(predictions[n],3)}\nExpected: {np.round(Yv[::898//scroll][n],3)} \n')

Predicted:[0.349 0.322]
Expected: [0.333 0.365] 

Predicted:[0.486 0.216]
Expected: [0.5   0.221] 

Predicted:[0.63 0.17]
Expected: [0.667 0.192] 

Predicted:[0.443 0.353]
Expected: [0.5   0.356] 

Predicted:[0.433 0.2  ]
Expected: [0.5   0.163] 

Predicted:[0.674 0.346]
Expected: [0.667 0.346] 

Predicted:[0.332 0.225]
Expected: [0.333 0.231] 

Predicted:[0.467 0.338]
Expected: [0.333 0.346] 

Predicted:[0.414 0.292]
Expected: [0.333 0.327] 

Predicted:[0.498 0.279]
Expected: [0.5   0.279] 

Predicted:[0.335 0.326]
Expected: [0.333 0.327] 

Predicted:[0.578 0.302]
Expected: [0.5   0.308] 

Predicted:[0.466 0.31 ]
Expected: [0.5   0.327] 

Predicted:[0.553 0.321]
Expected: [0.667 0.308] 

Predicted:[0.585 0.322]
Expected: [0.5   0.288] 

Predicted:[0.501 0.389]
Expected: [0.5   0.385] 

Predicted:[0.377 0.205]
Expected: [0.5   0.269] 

Predicted:[0.613 0.382]
Expected: [0.5   0.404] 

Predicted:[0.496 0.432]
Expected: [0.5   0.433] 

Predicted:[0.489 0.312]
Expected: [0.5   0.308] 

Pr